In [3]:
!nvidia-smi

Mon Jul 29 16:46:32 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.61                 Driver Version: 551.61         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   69C    P0             73W /  110W |    6732MiB /   8188MiB |     83%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
from torch import nn

torch.cuda.device_count()

1

In [3]:
torch.device('cpu'), torch.cuda.device('cuda')

(device(type='cpu'), <torch.cuda.device at 0x1f7fdd4d250>)

In [4]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [7]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.3068, 0.4259, 0.4958],
        [0.4966, 0.6200, 0.8791]], device='cuda:0')

In [8]:
Z = X + Y
Z

tensor([[1.3068, 1.4259, 1.4958],
        [1.4966, 1.6200, 1.8791]], device='cuda:0')

In [9]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)

tensor([[0.2903],
        [0.2903]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [10]:
net[0].weight.data.device

device(type='cuda', index=0)